# Introduction

## Goal. 
The goal of this lab is to continue the investigations of Evolutionary Algorithms (EAs) we started in the previous module's exercises. In particular, you will observe the effects of crossover, selection pressure, and population size in artificial evolution, and reflect to what extent these observations also apply to biological evolution.

This lab continues the use of the *inspyred* framework for the Python programming language seen in the previous lab. If you did not participate in the previous lab, you may want to look that over first and then start this lab's exercises.

Note once again that, unless otherwise specified, in this module's exercises we will use real-valued genotypes and that the aim of the algorithms will be to *minimize* the fitness function $f(\mathbf{x})$, i.e. lower values correspond to a better fitness!

# Exercise 1

In this exercise we will analyze the effect of crossover in the EA. An offspring individual is formed from two parent individuals $\mathbf{x}_1$ and $\mathbf{x}_2$ by randomly taking the value for each entry $x_i$ either from $\mathbf{x}_1$ or $\mathbf{x}_2$. The EA has a parameter defining the fraction of offspring that is created using crossover at each generation (the remaining individuals are created via asexual reproduction).

To start the experiments, run the next cell ${[1]}$.

This script executes $30$ runs using mutation only (as in the previous exercises), and $30$ runs using crossover only. The boxplots compare the best fitness values obtained in the two cases.

- Do you see any difference between the two results? Why?

---
[1]: 
For all the exercises in this lab you may set the seed for the pseudo-random number generator. This will allow you to reproduce your results. 


In [ ]:
import os
import sys

import numpy as np
from inspyred import benchmarks
from utils.simulation import run_ga_simulation, plot_boxplot

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
# parameters for the GA
args = {}
args["num_vars"] = 10  # Number of dimensions of the search space
args["gaussian_stdev"] = 1.0  # Standard deviation of the Gaussian mutations
args["tournament_size"] = 2
args["num_elites"] = 1  # number of elite individuals to maintain in each gen
args["pop_size"] = 20  # population size
args["pop_init_range"] = [-10, 10]  # Range for the initial population
args["max_generations"] = 50  # Number of generations of the GA

num_runs = 30  # Number of runs to be done for each condition

args["fig_title"] = f"GA mutation only std {args['gaussian_stdev']}"

# Only mutation, no crossover
args["mutation_rate"] = 1.0
args["crossover_rate"] = 0.0
# run the GA *num_runs* times and record the best fits
results = run_ga_simulation(
    func=benchmarks.Sphere,
    num_simulations=num_runs,
    args=args,
    print_plots=True,
)
# Display the results
print("Mean Best Individual (mutation only):", results.mean_best_individual)
print("Mean Best Fitness (mutation only):", results.mean_best_fitness)

# Only crossover, no mutation (uniform crossover)
args["mutation_rate"] = 0.0
args["crossover_rate"] = 1.0
# run the GA *num_runs* times and record the best fits
best_fitnesses_crossover_only = run_ga_simulation(
    func=benchmarks.Sphere,
    num_simulations=num_runs,
    args=args,
    print_plots=True,
)
# Display the results
print(
    "Mean Best Individual (crossover only):",
    best_fitnesses_crossover_only.mean_best_individual,
)
print(
    "Mean Best Fitness (crossover only):",
    best_fitnesses_crossover_only.mean_best_fitness,
)


args["mutation_rate"] = 0.3
args["crossover_rate"] = 0.6
results_mixed = run_ga_simulation(
    func=benchmarks.Sphere,
    num_simulations=num_runs,
    args=args,
    print_plots=True,
)
print("Mean Best Individual (mixed):", results_mixed.mean_best_individual)
print("Mean Best Fitness (mixed):", results_mixed.mean_best_fitness)

plot_boxplot(
    [
        results.all_best_fitness,
        best_fitnesses_crossover_only.all_best_fitness,
        results_mixed.all_best_fitness,
    ],
    ["Mutation only", "Crossover only", "Mixed"],
    "Strategy",
)

# Exercise 2

In this exercise we will focus on the effect of changing the fraction of offspring created using crossover. Run the next cell to compare the best fitnesses obtained by varying this fraction (while using a fixed mutation probability of $0.5$, i.e. each loci of each genome will have a $50\%$ chance of being mutated). 
- Is there an optimal crossover fraction for this fitness function? Why?


In [ ]:
from inspyred.benchmarks import Sphere

# parameters for the GA
args = {}
args["num_vars"] = 10  # Number of dimensions of the search space
args["gaussian_stdev"] = 1  # Standard deviation of the Gaussian mutations
args["mutation_rate"] = 0.5  # fraction of loci to perform mutation on
args["tournament_size"] = 2
args["num_elites"] = 1  # number of elite individuals to maintain in each gen
args["pop_size"] = 20  # population size
args["pop_init_range"] = [-10, 10]  # Range for the initial population
args["max_generations"] = 50  # Number of generations of the GA

num_runs = 30  # Number of runs to be done for each condition


crossover_rates = [0, 0.1, 0.25, 0.4, 0.75, 0.9, 1.0]
best_fitnesses = []
for crossover in crossover_rates:
    args["fig_title"] = f"GA crossover rate {crossover}"
    args["crossover_rate"] = crossover
    results = run_ga_simulation(
        func=Sphere,
        num_simulations=num_runs,
        args=args,
        print_plots=True,
    )
    best_fitnesses.append(results.all_best_fitness)

plot_boxplot(best_fitnesses, crossover_rates, "Crossover rate")

# Exercise 3

We will now investigate the effect of the selection pressure. In the previous exercises, we were using tournament selection with a tournament size of 2. Run the next cell to compare the best fitness values and the distances from the global optimum obtained using tournament sizes 2 and 10.

- Which tournament size gives better results for the fitness function sphere and why?
- Which tournament size is better for the fitness function __[Rastrigin] (https://pythonhosted.org/inspyred/reference.html?highlight=rastrigin#inspyred.benchmarks.Rastrigin)__ (higly multimodal, something like gradiant descent would get stuck) (you can change the problem by changing the parameter `problem_class` in the script) and why?



In [ ]:
# choose problem
problem_class = benchmarks.Sphere

# parameters for the GA
args = {}
args["num_vars"] = 10  # Number of dimensions of the search space
args["gaussian_stdev"] = 1  # Standard deviation of the Gaussian mutations
args["crossover_rate"] = 0.8  # Crossover fraction
args["mutation_rate"] = 1.0  # fraction of loci to perform mutation on
args["num_elites"] = 1  # number of elite individuals to maintain in each gen
args["pop_size"] = 25  # population size
args["pop_init_range"] = [-10, 10]  # Range for the initial population
args["max_generations"] = 100  # Number of generations of the GA

num_runs = 30  # Number of runs to be done for each condition


tournament_sizes = [2, 5, 10, 15, 20, 25]
best_fitnesses = []
for tournament_size in tournament_sizes:
    args["fig_title"] = "GA"
    args["tournament_size"] = tournament_size
    results = run_ga_simulation(
        func=problem_class,
        num_simulations=num_runs,
        args=args,
        print_plots=False,
    )
    best_fitnesses.append(results.all_best_fitness)
    print(f"Running with tournament size {tournament_size}")
    print("Mean Best Individual:", results.mean_best_individual)
    print("Mean Best Fitness:", results.mean_best_fitness)
    # The distance from the optimum in the N-dimensional space
    print(
        "Distance from Global Optimum",
        np.sqrt(np.sum(np.array(results.mean_best_individual) ** 2)),
    )

plot_boxplot(best_fitnesses, tournament_sizes, "Tournament size")

In [ ]:
# choose problem
problem_class = benchmarks.Rastrigin

# parameters for the GA
args = {}
args["num_vars"] = 10  # Number of dimensions of the search space
args["gaussian_stdev"] = 1  # Standard deviation of the Gaussian mutations
args["crossover_rate"] = 0.8  # Crossover fraction
args["mutation_rate"] = 1.0  # fraction of loci to perform mutation on
args["num_elites"] = 1  # number of elite individuals to maintain in each gen
args["pop_size"] = 25  # population size
args["pop_init_range"] = [-10, 10]  # Range for the initial population
args["max_generations"] = 100  # Number of generations of the GA

num_runs = 30  # Number of runs to be done for each condition


tournament_sizes = [2, 5, 10, 15, 20, 25]
best_fitnesses = []
for tournament_size in tournament_sizes:
    args["fig_title"] = "GA"
    args["tournament_size"] = tournament_size
    results = run_ga_simulation(
        func=problem_class,
        num_simulations=num_runs,
        args=args,
        print_plots=False,
    )
    best_fitnesses.append(results.all_best_fitness)
    print(f"Running with tournament size {tournament_size}")
    print("Mean Best Individual:", results.mean_best_individual)
    print("Mean Best Fitness:", results.mean_best_fitness)
    # The distance from the optimum in the N-dimensional space
    print(
        "Distance from Global Optimum",
        np.sqrt(np.sum(np.array(results.mean_best_individual) ** 2)),
    )

plot_boxplot(best_fitnesses, tournament_sizes, "Tournament size")

# Exercise 4

In this exercise you will run the EA on many test functions commonly used to benchmark optimization algorithms. Run the EA on some of the test functions shown in the comments of the next cell (especially the multimodal functions) and adapt the mutation magnitude, crossover rate, selection pressure, and population size so as to get the best results. If you run the code as provided it will initialize and bound the values of your population vectors to suitable ranges. You may comment/uncomment certain lines to alter this behavior. See the comments in the enxt cell for further details. 

You may first try the 1D or 2D case, which has the advantage that the fitness landscape can be visualized. However, keep in mind that sometimes the resolution of the plot is not sufficient to accurately represent a function.

- Do you see a different algorithmic behavior when you test the EA on different benchmark functions? Why?
- What is the effect of changing the number of variables on each tested function?


---
[1]:
See __[link](https://pythonhosted.org/inspyred/reference.html\#single-objective-benchmarks)__ for a list of single-objective benchmark problems.

In [ ]:
# Gets stuck in local minina really close to zero (fitness 0.039) with high standard deviation / larger probability of mutation we are able to escape this local minima
# parameters for the GA
args = {}
args["num_vars"] = 10  # Number of dimensions of the search space
args["gaussian_stdev"] = (
    0.1  # Standard deviation of the Gaussian mutations (initial 1.0)
)
args["crossover_rate"] = 0.8  # Crossover fraction
args["tournament_size"] = 2  # (initial 2)
args["pop_size"] = 10  # population size (initial 10)

args["num_elites"] = 1  # number of elite individuals to maintain in each gen
args["mutation_rate"] = 0.5  # fraction of loci to perform mutation on

# by default will use the problem's defined init_range
# uncomment the following line to use a specific range instead
# args["pop_init_range"] = [-500, 500] # Range for the initial population
args["use_bounder"] = True  # use the problem's bounder to restrict values
# comment out the previously line to run unbounded

args["max_generations"] = 100  # Number of generations of the GA
display = True  # Plot initial and final populations

# choose problem
problem_class = benchmarks.Sphere

# other problems to try,
# see  https://pythonhosted.org/inspyred/reference.html#module-inspyred.benchmarks

# unimodal
# problem_class = benchmarks.Rosenbrock

# multimodal
# problem_class = benchmarks.Griewank
# problem_class = benchmarks.Ackley
# problem_class = benchmarks.Rastrigin
# problem_class = benchmarks.Schwefel

# tecnically we shouldn't be able to find a set of parameters that work well for all benchmarks (NFL theorem)
"""
-------------------------------------------------------------------------
"""

args["fig_title"] = f"GA {problem_class.__name__}"

results = run_ga_simulation(
    func=problem_class,
    num_simulations=30,
    args=args,
    print_plots=display,
)
plot_boxplot([results.all_best_fitness], [""], "Fitness")
print("Mean Best Individual", results.mean_best_individual)
print("Mean Best Fitness", results.mean_best_fitness)

In [ ]:
# parameters for the GA
args = {}
args["num_vars"] = 10  # Number of dimensions of the search space
args["gaussian_stdev"] = (
    1.0  # Standard deviation of the Gaussian mutations (initial 1.0)
)
args["crossover_rate"] = 0.8  # Crossover fraction
args["tournament_size"] = 2
args["pop_size"] = 10  # population size (initial 10)

args["num_elites"] = (
    5  # number of elite individuals to maintain in each gen (initial 1)
)
args["mutation_rate"] = 0.1  # fraction of loci to perform mutation on

# by default will use the problem's defined init_range
# uncomment the following line to use a specific range instead
# args["pop_init_range"] = [-500, 500] # Range for the initial population
args["use_bounder"] = True  # use the problem's bounder to restrict values
# comment out the previously line to run unbounded

args["max_generations"] = 100  # Number of generations of the GA
display = True  # Plot initial and final populations

# choose problem
# problem_class = benchmarks.Sphere

# other problems to try,
# see  https://pythonhosted.org/inspyred/reference.html#module-inspyred.benchmarks

# unimodal
# problem_class = benchmarks.Rosenbrock

# multimodal
# problem_class = benchmarks.Griewank
problem_class = benchmarks.Ackley
# problem_class = benchmarks.Rastrigin
# problem_class = benchmarks.Schwefel

# tecnically we shouldn't be able to find a set of parameters that work well for all benchmarks (NFL theorem)

args["fig_title"] = f"GA {problem_class.__name__}"

results = run_ga_simulation(
    func=problem_class,
    num_simulations=30,
    args=args,
    print_plots=display,
)
plot_boxplot([results.all_best_fitness], [""], "Fitness")
print("Mean Best Individual", results.mean_best_individual)
print("Mean Best Fitness", results.mean_best_fitness)

In [ ]:
# parameters for the GA
args = {}
args["num_vars"] = 10  # Number of dimensions of the search space
args["gaussian_stdev"] = (
    0.1  # Standard deviation of the Gaussian mutations (initial 1.0)
)
args["crossover_rate"] = 0.2  # Crossover fraction
args["tournament_size"] = 2  # initial 2
args["pop_size"] = 25  # population size (initial 10)

args["num_elites"] = (
    0  # number of elite individuals to maintain in each gen (initial 1)
)
args["mutation_rate"] = 0.9  # fraction of loci to perform mutation on

# by default will use the problem's defined init_range
# uncomment the following line to use a specific range instead
# args["pop_init_range"] = [-500, 500] # Range for the initial population
args["use_bounder"] = True  # use the problem's bounder to restrict values
# comment out the previously line to run unbounded

args["max_generations"] = 100  # Number of generations of the GA
display = True  # Plot initial and final populations

# choose problem
# problem_class = benchmarks.Sphere

# other problems to try,
# see  https://pythonhosted.org/inspyred/reference.html#module-inspyred.benchmarks

# unimodal
# problem_class = benchmarks.Rosenbrock

# multimodal
# problem_class = benchmarks.Griewank
# problem_class = benchmarks.Ackley
problem_class = benchmarks.Rastrigin
# problem_class = benchmarks.Schwefel

# tecnically we shouldn't be able to find a set of parameters that work well for all benchmarks (NFL theorem)
args["fig_title"] = f"GA {problem_class.__name__}"

results = run_ga_simulation(
    func=problem_class,
    num_simulations=30,
    args=args,
    print_plots=display,
)
plot_boxplot([results.all_best_fitness], [""], "Fitness")
print("Mean Best Individual", results.mean_best_individual)
print("Mean Best Fitness", results.mean_best_fitness)

In [ ]:
# parameters for the GA
args = {}
args["num_vars"] = 10  # Number of dimensions of the search space
args["gaussian_stdev"] = (
    500.0  # Standard deviation of the Gaussian mutations (initial 1.0)
)
args["crossover_rate"] = 0.9  # Crossover fraction
args["tournament_size"] = 2
args["pop_size"] = 25  # population size (initial 10)

args["num_elites"] = (
    0  # number of elite individuals to maintain in each gen (initial 1)
)
args["mutation_rate"] = 0.01  # fraction of loci to perform mutation on

# by default will use the problem's defined init_range
# uncomment the following line to use a specific range instead
# args["pop_init_range"] = [-200, 200] # Range for the initial population
args["use_bounder"] = True  # use the problem's bounder to restrict values
# comment out the previously line to run unbounded

args["max_generations"] = 100  # Number of generations of the GA
display = True  # Plot initial and final populations

# choose problem
# problem_class = benchmarks.Sphere

# other problems to try,
# see  https://pythonhosted.org/inspyred/reference.html#module-inspyred.benchmarks

# unimodal
# problem_class = benchmarks.Rosenbrock

# multimodal
# problem_class = benchmarks.Griewank
# problem_class = benchmarks.Ackley
# problem_class = benchmarks.Rastrigin
problem_class = benchmarks.Schwefel

args["fig_title"] = f"GA {problem_class.__name__}"

results = run_ga_simulation(
    func=problem_class,
    num_simulations=30,
    args=args,
    print_plots=display,
)
plot_boxplot([results.all_best_fitness], [""], "Fitness")
print("Mean Best Individual", results.mean_best_individual)
print("Mean Best Fitness", results.mean_best_fitness)

## Instructions and questions

Concisely note down your observations from the previous exercises (follow the bullet points) and think about the following questions. 

- Why is it useful to introduce crossover in EA? Can you think of any cases when mutation only can work effectively, without crossover? 
What about using crossover only, without mutation?
- What's the effect of changing the fraction of offspring created by crossover?
- Are there optimal parameters for an EA?
What are the advantages and disadvantages of low/high selection pressure?